# Data Analyst Skills Analysis: Data Cleaning and Skills Engineering

## 0.1 Project goals and objectives

**Goal:**  
To clean and prepare job posting data from the JSearch API (data analyst and junior data analyst roles across major English-speaking markets) so that skill requirements can be extracted and analysed in a follow-up exploratory analysis.

**Key objectives:**
- Load and explore the combined raw dataset from the JSearch data collection step;
- Perform data preprocessing (column standardisation, data type conversion, missing values, duplicates, categorical validation);
- Engineer additional features such as `experience_group` and structured text fields derived from `highlights`;
- Clean job title, description, and highlights text for skill extraction;
- Define a comprehensive skills dictionary with regex patterns for keyword-based extraction;
- Extract skill indicators using the combined text fields.

## 0.2 Data description

The project uses job posting data obtained via the **JSearch API** (RapidAPI) for the following markets and roles:

- **Countries (search scope):** USA (`us`), United Kingdom (`gb`), Canada (`ca`);
- **Roles (search queries):** *data analyst*, *junior data analyst*;
- **Time window:** only postings from the last month are included (18 Jan – 17 Feb 2026).

The raw **jsearch_all_countries_roles.csv** dataset contains one row per job posting. Main columns:

- **job_id** — unique job identifier returned by JSearch;
- **job_title** — job title;
- **job_description** — full job description text;
- **employer_name** — company name;
- **employer_website** — company website (if available);
- **job_publisher** — job board or source where the posting was found;
- **job_employment_type** — type of employment (e.g. Full-time, Part-time, Contractor);
- **job_is_remote** — Boolean flag indicating whether the role is remote;
- **job_apply_link** — application URL;
- **job_city, job_state, job_country** — location information parsed by the API;
- **job_latitude, job_longitude** — geographic coordinates (where available);
- **job_posted_at_datetime_utc** — publication date and time in UTC;
- **job_min_salary, job_max_salary, job_salary_period** — salary information where available;
- **job_highlights** — structured highlights returned by the API (e.g. Qualifications, Responsibilities);
- **country_code, country_name** — country metadata based on the search configuration;
- **search_role** — search query used when fetching the job (*data analyst* or *junior data analyst*);
- **data_source** — source tag (here, always `jsearch`).

## 0.3 Project structure

- [1. Data loading and overview](#section-1)
- [2. Data preprocessing](#section-2)
- [3. Feature engineering](#section-3)
- [4. Cleaning title and description fields](#section-4)
- [5. Defining a skills dictionary](#section-5)
- [6. Extracting skills from job descriptions](#section-6)
- [7. Summary](#section-7)

<a id="section-1"></a>

## 1. Data loading and overview

In [1]:
import os
import re
import numpy as np
import pandas as pd
import ast

In [2]:
df = pd.read_csv("../data/raw/jsearch_all_countries_roles.csv")

df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 23 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   job_id                      1228 non-null   str    
 1   job_title                   1228 non-null   str    
 2   job_description             1228 non-null   str    
 3   employer_name               1228 non-null   str    
 4   employer_website            917 non-null    str    
 5   job_publisher               1228 non-null   str    
 6   job_employment_type         1209 non-null   str    
 7   job_is_remote               1228 non-null   bool   
 8   job_apply_link              1228 non-null   str    
 9   job_city                    928 non-null    str    
 10  job_state                   898 non-null    str    
 11  job_country                 1195 non-null   str    
 12  job_latitude                1195 non-null   float64
 13  job_longitude               1195 non-null   

In [3]:
df.head()

,job_id,job_title,job_description,employer_name,employer_website,job_publisher,job_employment_type,job_is_remote,job_apply_link,job_city,...,job_longitude,job_posted_at_datetime_utc,job_min_salary,job_max_salary,job_salary_period,job_highlights,country_code,country_name,search_role,data_source
0,hDcRNAz4ev12li_ZAAAAAA==,Data Analyst,Join a newly created team dedicated to the Dis...,Disney Direct to Consumer,https://disney.fandom.com,Disney Careers,Full-time,False,https://www.disneycareers.com/en/job/new-york/...,New York,...,-74.005973,2026-02-12T00:00:00.000Z,NaN,NaN,NaN,{'Qualifications': ['3+ years of relevant expe...,us,USA,data analyst,jsearch
1,YBGg4U6cLzfYcc2VAAAAAA==,Entry Level Human Resources & Data Analyst,"Top-Tier Bank in Midtown, Manhattan is seeking...",Social Capital Resources,NaN,LinkedIn,Contractor,False,https://www.linkedin.com/jobs/view/entry-level...,New York,...,-74.005973,2026-02-17T17:00:00.000Z,25.00,32.0,HOUR,{'Qualifications': ['0-1 years previous experi...,us,USA,data analyst,jsearch
2,wyxH_fGPJReWty01AAAAAA==,"Associate Data Analyst, DX Research",Overview\n\nAbout DX\n\nDX is one of the faste...,Atlassian,https://www.atlassian.com,LinkedIn,Full-time,True,https://www.linkedin.com/jobs/view/associate-d...,New York,...,-74.005973,2026-02-14T00:00:00.000Z,NaN,NaN,NaN,{'Qualifications': ['4+ years of experience in...,us,USA,data analyst,jsearch
3,GfD-wE5qzfVrU4rkAAAAAA==,Data Analyst I - Artificial Intelligence & Hum...,Description\n\nThe Data Analyst I is responsib...,Mount Sinai Health Systems,https://www.mountsinai.org,Mount Sinai Careers,Full-time,False,https://careers.mountsinai.org/jobs/3034743?la...,New York,...,-74.005973,2026-02-08T00:00:00.000Z,66482.07,74250.0,YEAR,{'Qualifications': ['Bachelors degree in compu...,us,USA,data analyst,jsearch
4,0XEjS66pv9q6bLyWAAAAAA==,Data Analyst TC - Data and Analytics - DAE - F...,EY focuses on high-ethical standards and integ...,EY,NaN,Careers At EY,Full-time,False,https://careers.ey.com/ey/job/New-York-Data-An...,New York,...,-74.005973,2026-02-09T00:00:00.000Z,NaN,NaN,NaN,"{'Qualifications': [""Must have a Bachelor's de...",us,USA,data analyst,jsearch


The combined dataset contains **23 columns and 1228 rows**:
- The data matches the expected structure.
- The `job_posted_at_datetime_utc` column should be converted to a datetime type. Other columns use appropriate data types.
- Missing values are present in the salary fields (`job_salary_max`, `job_salary_min`,`job_salary_period`), location fields (`job_longitude`,`job_latitude`,`job_country`,`job_state`,`job_city`), `job_employment_type`, and `employer_website` columns.

<a id="section-2"></a>

## 2. Data preprocessing

### 2.1. Standardization of columns

We standardize column names to make the dataframe easier to work with:

In [4]:
df.columns = [col.replace("job_", "") if col.startswith("job_") else col for col in df.columns]

In [5]:
df.info()

<class 'pandas.DataFrame'>
RangeIndex: 1228 entries, 0 to 1227
Data columns (total 23 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1228 non-null   str    
 1   title                   1228 non-null   str    
 2   description             1228 non-null   str    
 3   employer_name           1228 non-null   str    
 4   employer_website        917 non-null    str    
 5   publisher               1228 non-null   str    
 6   employment_type         1209 non-null   str    
 7   is_remote               1228 non-null   bool   
 8   apply_link              1228 non-null   str    
 9   city                    928 non-null    str    
 10  state                   898 non-null    str    
 11  country                 1195 non-null   str    
 12  latitude                1195 non-null   float64
 13  longitude               1195 non-null   float64
 14  posted_at_datetime_utc  1228 non-null   str    
 15

### 2.2 Data type conversion

In [6]:
df['posted_at_datetime_utc'] = pd.to_datetime(df['posted_at_datetime_utc'])

### 2.3. Missing values analysis

In [7]:
missing_summary = pd.DataFrame({
    'missing_count': df.isna().sum(),
    'missing_share': df.isna().mean()
}).sort_values(by='missing_count', ascending=False)

display(missing_summary)

,missing_count,missing_share
min_salary,1016,0.827362
max_salary,1016,0.827362
salary_period,847,0.689739
state,330,0.268730
employer_website,311,0.253257
city,300,0.244300
country,33,0.026873
latitude,33,0.026873
longitude,33,0.026873
employment_type,19,0.015472


Most missing values are concentrated in location and salary fields: `state` (26.9%), `city` (24.4%), `country` (2.7%), `min_salary`/`max_salary` (~82.7% each), `salary_period` (~69.0%). Smaller gaps occur in `employment_type` (~1.5%) and `employer_website` (~25.3%). 

These patterns are consistent with how job ads are published: employers often omit city/state details for remote or flexible roles and do not always disclose salary information. Since the main focus of this project is on skill requirements rather than precise geography or salary modelling, these missing values are considered acceptable and are left as is.

### 2.4. Detection of explicit and implicit duplicates

In [8]:
print (f'Total number of duplicate records: {df.duplicated().sum()}')
print (f'Share of duplicate records: {(df.duplicated().sum()/ df.shape[0])}')

Total number of duplicate records: 0
Share of duplicate records: 0.0


In [9]:
for column in ['publisher','employment_type','salary_period','search_role','data_source','country_code','country_name','country','state','city']:
    print(f'\nNumber of unique values in the `{column}` column: {df[column].nunique()}')
    print(f'\nUnique values in the `{column}` column: {df[column].sort_values().unique()}')


Number of unique values in the `publisher` column: 233

Unique values in the `publisher` column: <StringArray>
[                      '@Artand.services',
                          'ACG Resources',
                              'Accenture',
                   'AdventHealth Careers',
                                 'Adzuna',
               'Alvarez & Marsal Careers',
                        'Amentum Careers',
                                'Apply4U',
                'Aspire Media Group Jobs',
                   'Athenahealth Careers',
 ...
                       'Women's Job List',
 'Worcester Regional Chamber Of Commerce',
                               'Workable',
                          'Workable Jobs',
                                'Workday',
                             'Workopolis',
                           'ZipRecruiter',
                                 'Zippia',
                             'reed.co.uk',
                                    '海拉拉']
Length: 233, dtype: str

The analysis of categorical variables revealed potential inconsistencies between the `country_code` metadata and the `country` values returned by the API (e.g. postings fetched under a US query but tagged with non-US country codes). To better understand this issue, we first count how many rows exhibit such mismatches:

In [10]:
mismatches = df[df["country"].notna() & (df["country"].str.upper() != df["country_code"].str.upper())][["country_code", "country_name", "country"]]

mismatches.value_counts().head(20)

country_code  country_name    country
gb            United Kingdom  IN         4
                              MX         3
                              DE         3
us            USA             GB         2
gb            United Kingdom  MY         2
                              NG         2
                              FR         2
                              US         2
us            USA             CA         1
gb            United Kingdom  SK         1
                              CY         1
                              SG         1
                              HR         1
                              PH         1
                              ID         1
                              CA         1
ca            Canada          SG         1
                              NG         1
                              IN         1
Name: count, dtype: int64

We remove rows with inconsistent country information (where the `country_code` used for collection does not match the `country` reported by the API) to ensure consistency in the geographic scope of the analysis:

In [11]:
mismatch_mask = df["country"].notna() & ((df["country"].str.upper() != df["country_code"].str.upper()))

print(f"Rows with mismatched country values: {mismatch_mask.sum()}")

df = df[~mismatch_mask].copy()

Rows with mismatched country values: 31


Next, the `employment_type` field contains a mix of base categories (Full-time, Part-time, Contractor, Internship) and combined values (e.g. "Full-time and Contractor"), as well as minor inconsistencies in formatting (different dash characters). To make this variable easier to analyse, we normalise the text and create separate binary flags for each main employment type:

In [12]:
df["employment_type"] = df["employment_type"].str.replace("–", "-", regex=False).str.lower()

df["is_full_time"] = df["employment_type"].str.contains("full-time", na=False).astype(int)
df["is_part_time"] = df["employment_type"].str.contains("part-time", na=False).astype(int)
df["is_contractor"] = df["employment_type"].str.contains("contractor", na=False).astype(int)
df["is_internship"] = df["employment_type"].str.contains("internship", na=False).astype(int)

In [13]:
df["employment_type"].value_counts()

employment_type
full-time                              1006
contractor                               70
internship                               29
part-time                                27
full-time and contractor                 22
full-time and part-time                  19
full-time and internship                  4
full-time, part-time and contractor       1
Name: count, dtype: int64

The remaining categorical variables appear to be valid and consistent. Next, we check for implicit duplicates:

In [14]:
df.duplicated(subset=['title','description', 'employer_name', 'posted_at_datetime_utc']).sum()

np.int64(59)

In [15]:
df[df.duplicated(subset=['title','description', 'employer_name', 'posted_at_datetime_utc'], keep=False)].sort_values(by='employer_name')

,id,title,description,employer_name,employer_website,publisher,employment_type,is_remote,apply_link,city,...,salary_period,highlights,country_code,country_name,search_role,data_source,is_full_time,is_part_time,is_contractor,is_internship
465,sB8L6RlZpcDAvURnAAAAAA==,Jr. Project Manager / Data Analyst,**Job Description**\n\nIn this role you will p...,"ASM Research, An Accenture Federal Services Co...",NaN,Adzuna,full-time,False,https://www.adzuna.com/details/5594844835?utm_...,Boston,...,NaN,{'Qualifications': ['If you are a continuous l...,us,USA,data analyst,jsearch,1,0,0,0
642,Fdw18ECvpGEfRa7NAAAAAA==,Jr. Project Manager / Data Analyst,**Job Description**\n\nIn this role you will p...,"ASM Research, An Accenture Federal Services Co...",NaN,Adzuna,full-time,False,https://www.adzuna.com/details/5594843762?utm_...,Denver,...,NaN,{'Qualifications': ['If you are a continuous l...,us,USA,data analyst,jsearch,1,0,0,0
59,ZKkShXjn47eyDCzPAAAAAA==,"Data Analyst, Data Analytics",**What Data Analytics brings to Cardinal Healt...,Cardinal Health,https://www.cardinalhealth.com,Adzuna,full-time,False,https://www.adzuna.com/details/5626718857?utm_...,Sacramento,...,NaN,{'Qualifications': ['Data Analytics is respons...,us,USA,data analyst,jsearch,1,0,0,0
195,z9uV7TCPRuSibZUrAAAAAA==,"Data Analyst, Data Analytics",**What Data Analytics brings to Cardinal Healt...,Cardinal Health,https://www.cardinalhealth.com,Adzuna,full-time,False,https://www.adzuna.com/details/5626742957?utm_...,Springfield,...,NaN,{'Qualifications': ['Data Analytics is respons...,us,USA,data analyst,jsearch,1,0,0,0
156,MVFgQFtK8E6NFiEWAAAAAA==,Licensed Vocational Nurse Clinical Data Analys...,DescriptionCHRISTUS Spohn Hospital Kleberg is ...,Christus Health,https://www.christushealth.org,Adzuna,full-time,False,https://www.adzuna.com/details/5627774511?utm_...,Riviera,...,NaN,{'Qualifications': ['Graduate from an accredit...,us,USA,data analyst,jsearch,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138,kflFjBjtFLCJ97H9AAAAAA==,Data Analyst - Power Platform,A company is looking for a Data Analyst - Powe...,VirtualVocations,https://www.virtualvocations.com,Talent.com,full-time,False,https://www.talent.com/view?id=29529897e413&ut...,Beaumont,...,NaN,{'Qualifications': ['8 years of experience in ...,us,USA,data analyst,jsearch,1,0,0,0
323,gJtQoIOsqhrs3dwBAAAAAA==,Senior Operations Data Analyst,A company is looking for a Senior Operations D...,VirtualVocations,https://www.virtualvocations.com,Talent.com,full-time,False,https://www.talent.com/view?id=5fd2aa45585e&ut...,West Palm Beach,...,NaN,{'Qualifications': ['Advanced proficiency in S...,us,USA,data analyst,jsearch,1,0,0,0
399,8iIRySXnyqFcJd0eAAAAAA==,Data Analyst - Power Platform,A company is looking for a Data Analyst - Powe...,VirtualVocations,https://www.virtualvocations.com,Talent.com,full-time,False,https://www.talent.com/view?id=cef8a3bd78e6&ut...,Portsmouth,...,NaN,{'Qualifications': ['8 years of experience in ...,us,USA,data analyst,jsearch,1,0,0,0
832,nX0O9-TDerOUztA9AAAAAA==,NCFE Data Analyst Level 4 Tutor,Reports to: Module Lead/Programme Manager\n\nH...,Waltham international College,NaN,Indeed,full-time and contractor,False,https://uk.indeed.com/viewjob?jk=ecb2519c2aa0a...,NaN,...,YEAR,{},gb,United Kingdom,data analyst,jsearch,1,0,1,0


We found 59 job postings with identical `title`, `description`, `employer_name` and creation time - duplicates likely created during parsing. We will remove them:

In [16]:
df = df.drop_duplicates(subset=['title','description', 'employer_name', 'posted_at_datetime_utc'])

<a id="section-3"></a>

## 3. Feature engineering

### 3.1 Deriving experience level


To analyse skill requirements by seniority, we derive an `experience_group` variable from the job title rather than relying only on the original search queries. The `search_role` column reflects how vacancies were fetched from the API (`"data analyst"` or `"junior data analyst"`), while `experience_group` captures how employers actually position the role in the title text.

We map each title directly into two broad groups:
- `junior` — titles mentioning entry-level, graduate, intern, trainee, or junior (including common abbreviations like "jr");
- `mid_plus` — all other roles without explicit junior/entry markers.

In practice, search queries and title-based seniority do not always align: a `"data analyst"` search can return senior or lead positions, and a `"junior data analyst"` search may include generic "Data Analyst" titles. We keep both signals to be transparent about this mismatch, but use `experience_group` as the main variable when comparing skill requirements between junior and more experienced roles.

In [17]:
def get_experience_group(title: str) -> str:
    """
    Map job titles directly into two broad groups:
    - junior: titles mentioning entry-level, graduate, intern, trainee, or junior;
    - mid_plus: all other roles.
    """
    if not isinstance(title, str):
        return "mid_plus"

    t = title.lower()

    # Junior / entry-like signals
    if (
        "entry level" in t
        or "entry-level" in t
        or "graduate" in t
        or "intern" in t
        or "trainee" in t
        or "junior" in t
        or "jr " in t
        or "jr." in t
    ):
        return "junior"

    # Everything else
    return "mid_plus"


df["experience_group"] = df["title"].apply(get_experience_group)
df["experience_group"].value_counts()

experience_group
mid_plus    852
junior      286
Name: count, dtype: int64

### 3.2 Deriving structured text from `highlights`

The `highlights` field contains semi-structured information returned by the API, typically as a dictionary with lists of bullet points (e.g. `{"Qualifications": [...], "Responsibilities": [...]}`). To make this information easier to use in the analysis, we extract two additional text fields:

- `qualifications_text` — a single string created by concatenating all entries under the `Qualifications` key;
- `responsibilities_text` — a single string created by concatenating all entries under the `Responsibilities` key.

In [18]:
def extract_highlights_text(highlights_str: str) -> tuple[str, str]:
    """
    Parse the 'highlights' field (stringified dict) and extract
    concatenated text for Qualifications and Responsibilities.
    """
    if not isinstance(highlights_str, str):
        return "", ""
    
    try:
        data = ast.literal_eval(highlights_str)
    except (ValueError, SyntaxError):
        return "", ""
    
    # Each value in the dict is usually a list of strings
    quals_list = data.get("Qualifications", []) or []
    resp_list = data.get("Responsibilities", []) or []
    
    quals_text = " ".join(quals_list)
    resp_text = " ".join(resp_list)
    
    return quals_text, resp_text


df["qualifications_text"], df["responsibilities_text"] = zip(
    *df["highlights"].apply(extract_highlights_text)
)

<a id="section-4"></a>

## 4. Cleaning title and description fields

For skill extraction and text analysis we create cleaned versions of the text fields: we lowercase the text, remove basic HTML tags and extra whitespace. 

We keep the original raw text columns and store the cleaned text in new columns: `title_clean`, `description_clean`,`qualifications_clean`,`responsibilities_clean`.

In [19]:
def basic_clean_text(text: str) -> str:
    """
    Perform a basic text cleaning:
    - convert to lowercase,
    - remove simple HTML tags,
    - strip extra whitespace.
    """
    if not isinstance(text, str):
        return ""

    text = re.sub(r"<.*?>", " ", text)

    text = text.lower()

    text = re.sub(r"\s+", " ", text).strip()

    return text

df["title"] = df["title"].apply(basic_clean_text)
df["description"] = df["description"].apply(basic_clean_text)
df["qualifications_text"] = df["qualifications_text"].apply(basic_clean_text)
df["responsibilities_text"] = df["responsibilities_text"].apply(basic_clean_text)

df[["title", "description", "qualifications_text", "responsibilities_text"]].head()

,title,description,qualifications_text,responsibilities_text
0,data analyst,join a newly created team dedicated to the dis...,3+ years of relevant experience strong program...,as part of disney's rapidly evolving streaming...
1,entry level human resources & data analyst,"top-tier bank in midtown, manhattan is seeking...",0-1 years previous experience in human resourc...,this person will mainly working in data and be...
2,"associate data analyst, dx research",overview about dx dx is one of the fastest-gro...,4+ years of experience in an analytical role (...,you’ll report into the marketing team and be r...
3,data analyst i - artificial intelligence & hum...,description the data analyst i is responsible ...,"bachelors degree in computer science, statisti...",the data analyst i is responsible for implemen...
4,data analyst tc - data and analytics - dae - f...,ey focuses on high-ethical standards and integ...,"must have a bachelor's degree in engineering, ...",understand business challenges and address rem...


<a id="section-5"></a>

## 5. Defining a skills dictionary

Next, we define a skills dictionary that maps each canonical skill name
(e.g. "sql", "excel", "power_bi") to a list of regex patterns or keywords that we will search for in the job descriptions.  

This dictionary will be used later for keyword-based skill extraction.

We use a hybrid dictionary: each skill maps to several regex patterns, including synonyms and typical phrases (e.g. "relational databases", "building dashboards", "data viz").

In [20]:
# Hybrid dictionary: keywords + synonyms and typical phrases (description_clean is lowercased)
skills_dict = {
    # Programming and query languages
    "sql": [
        r"\bsql\b",
        r"\brelational database(s)?\b",
        r"\bsql quer(y|ies)\b",
        r"\bwriting queries\b",
        r"\bquerying (databases|data)\b",
        r"\bwrite sql\b",
    ],
    "python": [
        r"\bpython\b",
        r"\bpython programming\b",
        r"\bproficiency in python\b",
    ],
    "r": [r"\br language\b", r"\br/\b", r"\br programming\b", r"\br studio\b", r"\br\b"],
    
    # Spreadsheets
    "excel": [
        r"\bexcel\b",
        r"\bms excel\b",
        r"\bmicrosoft excel\b",
        r"\badvanced excel\b",
        r"\bproficiency in excel\b",
    ],
    "google_sheets": [r"\bgoogle sheets\b", r"\bgoogle spreadsheet(s)?\b"],
    
    # Traditional databases
    "postgresql": [r"\bpostgres\b", r"\bpostgresql\b"],
    "mysql": [r"\bmysql\b"],
    "oracle": [r"\boracle\b", r"\boracle database\b"],
    "sql_server": [r"\bsql server\b", r"\bmssql\b", r"\bmicrosoft sql server\b"],
    
    # Modern data warehouses and cloud platforms
    "bigquery": [r"\bbigquery\b", r"\bgoogle bigquery\b"],
    "snowflake": [r"\bsnowflake\b"],
    "redshift": [r"\bredshift\b", r"\bamazon redshift\b", r"\baws redshift\b"],
    "vertica": [r"\bvertica\b"],
    "clickhouse": [r"\bclickhouse\b"],
    "databricks": [r"\bdatabricks\b"],
    "synapse": [r"\bazure synapse\b", r"\bsynapse\b"],
    
    # BI tools (variants: power bi, powerbi, power-bi)
    "power_bi": [r"\bpower\s*[- ]?bi\b", r"\bpowerbi\b"],
    "tableau": [r"\btableau\b"],
    "looker": [r"\blooker\b"],
    "qlik": [r"\bqlik\b", r"\bqlikview\b", r"\bqliksense\b"],
    "datalens": [r"\bdatalens\b", r"\byandex datalens\b"],
    "superset": [r"\bsuperset\b", r"\bapache superset\b"],
    
    # Data engineering and ETL tools
    "airflow": [r"\bairflow\b", r"\bapache airflow\b"],
    "hadoop": [r"\bhadoop\b", r"\bapache hadoop\b"],
    "spark": [r"\bspark\b", r"\bapache spark\b", r"\bpyspark\b"],
    "kafka": [r"\bkafka\b", r"\bapache kafka\b"],
    "dbt": [r"\bdbt\b", r"\bdata build tool\b"],
    "talend": [r"\btalend\b"],
    "informatica": [r"\binformatica\b"],
    
    # AI/ML tools and assistants
    "chatgpt": [r"\bchatgpt\b", r"\bchat gpt\b"],
    "claude": [r"\bclaude\b", r"\banthropic claude\b"],
    "cursor": [r"\bcursor\b", r"\bcursor ai\b"],
    "copilot": [r"\bcopilot\b", r"\bgithub copilot\b"],
    "gemini": [r"\bgemini\b", r"\bgoogle gemini\b"],
    
    # Analytics & statistics
    "statistics": [
        r"\bstatistics\b",
        r"\bstatistical\b",
        r"\bstatistical analysis\b",
        r"\bstatistical methods\b",
    ],
    "a_b_testing": [r"\ba/b testing\b", r"\ba b testing\b", r"\ba/b test(s)?\b", r"\bab testing\b"],
    "experimentation": [r"\bexperimentation\b", r"\bexperiments\b", r"\brun experiments\b"],
    "hypothesis_testing": [r"\bhypothesis testing\b", r"\bhypothesis test(s)?\b"],
    
    # Data workflows
    "etl": [
        r"\betl\b",
        r"\bextract transform load\b",
        r"\betl process(es)?\b",
        r"\betl pipeline(s)?\b",
    ],
    "data_cleaning": [r"\bdata cleaning\b", r"\bdata cleansing\b", r"\bcleaning data\b"],
    "data_pipeline": [
        r"\bdata pipeline(s)?\b",
        r"\bpipelines\b",
        r"\bdata pipeline development\b",
    ],
    
    # Visualization / dashboards
    "dashboards": [
        r"\bdashboard(s)?\b",
        r"\bbuilding dashboard(s)?\b",
        r"\bcreate dashboard(s)?\b",
        r"\bdashboard development\b",
        r"\bdeveloping dashboard(s)?\b",
    ],
    "data_visualization": [
        r"\bdata visualization\b",
        r"\bdata visualisation\b",
        r"\bdata viz\b",
        r"\bvisualizing data\b",
        r"\bdata visualizations\b",
    ],
}

print(f"Total number of skills in dictionary: {len(skills_dict)}")

Total number of skills in dictionary: 43


<a id="section-6"></a>

## 6. Extracting skills from job descriptions

To capture as many relevant skill mentions as possible, we first create a `text_for_skills` field by concatenating three cleaned text sources for each posting: job description, qualifications, and responsibilities. 

We then create a helper function `text_contains_any_pattern` to check whether any of the regex patterns defined in `skills_dict` appear in this combined text. 

For each skill, we create a binary indicator column (`skill_<name>`) showing whether the skill is mentioned in the posting. 

Finally, we run a quick check to see how many postings mention each skill and to verify that the extraction works as expected.

In [21]:
df["text_for_skills"] = (
    df["description"].fillna("")
    + " "
    + df["qualifications_text"].fillna("")
    + " "
    + df["responsibilities_text"].fillna("")
).str.strip()

In [22]:
def text_contains_any_pattern(text: str, patterns: list) -> bool:
    """
    Return True if the given text contains any of the regex patterns.
    """
    if not isinstance(text, str):
        return False

    for pattern in patterns:
        if re.search(pattern, text):
            return True
    return False

for skill_name, patterns in skills_dict.items():
    col_name = f"skill_{skill_name}"
    df[col_name] = df["text_for_skills"].apply(
        lambda txt: int(text_contains_any_pattern(txt, patterns))
    )

In [23]:
# Quick check: how many postings mention SQL, Python, etc.
skill_cols = [col for col in df.columns if col.startswith("skill_")]
df[skill_cols].sum().sort_values(ascending=False)

skill_sql                   628
skill_python                413
skill_excel                 409
skill_dashboards            394
skill_power_bi              357
skill_statistics            316
skill_tableau               288
skill_data_visualization    266
skill_r                     195
skill_data_pipeline         122
skill_etl                    95
skill_snowflake              90
skill_looker                 71
skill_spark                  51
skill_experimentation        48
skill_oracle                 48
skill_databricks             47
skill_sql_server             45
skill_dbt                    42
skill_a_b_testing            42
skill_data_cleaning          39
skill_bigquery               29
skill_redshift               24
skill_google_sheets          22
skill_synapse                18
skill_qlik                   16
skill_hadoop                 15
skill_informatica            13
skill_postgresql             11
skill_airflow                 9
skill_hypothesis_testing      8
skill_ka

In [24]:
PROCESSED_DIR = "../data/processed"
os.makedirs(PROCESSED_DIR, exist_ok=True)

processed_path = os.path.join(PROCESSED_DIR, "jsearch_cleaned_with_skills.csv")
df.to_csv(processed_path, index=False)

print(f"Processed dataset saved to: {processed_path}")

Processed dataset saved to: ../data/processed/jsearch_cleaned_with_skills.csv


<a id="section-7"></a>

## 7. Summary

During the data cleaning and feature-engineering stage, the following steps were performed:

- **Data loading:** The combined raw JSearch dataset `jsearch_all_countries_roles.csv` was loaded (1,228 rows, 23 columns) for data analyst and junior data analyst roles in the USA, UK, and Canada.
- **Column standardisation:** Column names were harmonised by removing the `job_` prefix and renaming fields to a more concise and consistent format.
- **Data type conversion and missing values:** The `posted_at_datetime_utc` column was converted to `datetime64`. Most missing values were found in salary and location fields, which were left as is given the focus on skill requirements.
- **Categorical checks and cleaning:** Key categorical variables were inspected. Rows with inconsistent country information (where `country_code` did not match the API-reported `country`) were identified and removed to keep the geographic scope consistent. The `employment_type` field was normalised and expanded into binary flags for full-time, part-time, contractor, and internship roles.
- **Duplicate removal:** Implicit duplicates (identical title, description, employer, and timestamp) were detected and removed.
- **Feature engineering:** An `experience_group` variable was derived from job titles to distinguish between junior (entry-level and junior roles) and more experienced positions (`mid_plus`). Additionally, structured highlights from the API were parsed into separate text fields, `qualifications_text` and `responsibilities_text`.
- **Text cleaning:** Cleaned versions of the title, description, qualifications, and responsibilities fields were created (lowercased, HTML tags removed, extra whitespace normalised). The three text sources were then combined into a single `text_for_skills` field to maximise skill detection coverage.
- **Skill extraction:** A skills dictionary was defined, mapping 43 canonical skill names to regex patterns. A helper function `text_contains_any_pattern` was applied to `text_for_skills` to create binary indicator columns (`skill_<name>`) for each skill.

After all cleaning steps, the dataset went from 1,228 to 1,138 rows (~7.3% removed). 

The preprocessed dataset was saved as `jsearch_cleaned_with_skills.csv` in `data/processed/` and is ready for use in the exploratory analysis and visualisation notebook.